<a href="https://colab.research.google.com/github/shashithenuwara/IRWA_Project/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install PyPDF2 pdfplumber spacy nltk whoosh transformers torch beautifulsoup4
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import os
from PyPDF2 import PdfReader

#import files to the colab environment
pdf_directory = '/content/drive/My Drive/Colab Notebooks/MIniDataSet'

# extract texts from all PDFs
text_data = ""
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text_data += page.extract_text() + "\n"




In [50]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')

# Tokenization
tokens = word_tokenize(text_data)

# Normalization (Lowercasing, Removing punctuation, Stopwords)
tokens = [word.lower() for word in tokens if word.isalpha()]
tokens = [word for word in tokens if word not in stopwords.words('english')]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
import spacy

nlp = spacy.load('en_core_web_sm')

# Increase the max_length limit
nlp.max_length = len(text_data)

doc = nlp(text_data)

for ent in doc.ents:
    print(ent.text, ent.label_)

Streaming output truncated to the last 5000 lines.
1 CARDINAL
Experimental Design ORG
NAC ORG
zero CARDINAL
two CARDINAL
NAC ORG
NAC ORG
3 CARDINAL
RNA ORG
ITH ORG
TNBC ORG
NAC ORG
two CARDINAL
NAC ORG
alterna GPE
NAC ORG
RNA ORG
RESULTS ORG
Patients Treated with Neoadjuvant
Chemotherapy WORK_OF_ART
20 CARDINAL
TNBC ORG
NAC ORG
S1 PRODUCT
TNBC ORG
ER ORG
HER2 PERSON
NAC ORG
two CARDINAL
four CARDINAL
1 CARDINAL
Frozen PERSON
two CARDINAL
NAC ORG
two CARDINAL
880 FAC
879–893 CARDINAL
May 3, 2018 DATE
20 CARDINAL
RNA ORG
eight CARDINAL
Clonal Extinction ORG
Response ORG
NAC ORG
180 CARDINAL
3)o CARDINAL
20 CARDINAL
Table S2 andSTAR Methods LAW
60% PERCENT
TNBC ORG
TCGA ORG
2012 DATE
49 CARDINAL
3 CARDINAL
1–6 CARDINAL
20 CARDINAL
10 CARDINAL
2 CARDINAL
10 CARDINAL
2 CARDINAL
NAC ORG
TNBC ORG
2 CARDINAL
Roth PERSON
al. GPE
2014 DATE
Malikic et al. PERSON
2015 DATE
Sathirapongsasuti GPE
al. GPE
2011 DATE
al. GPE
2014 DATE
2 CARDINAL
2D ORG
two to four CARDINAL
TNBC ORG
two CARDINAL
1 CARDI

In [52]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Stemming
stemmer = PorterStemmer()
stems = [stemmer.stem(word) for word in tokens]

# Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmas = [lemmatizer.lemmatize(word) for word in tokens]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [53]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
import os

# Define schema
schema = Schema(title=TEXT(stored=True), content=TEXT)

# Create index directory
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

# Create an index
ix = create_in("indexdir", schema)

# Indexing documents (Assuming titles are available for each document)
writer = ix.writer()

# Add documents to index
writer.add_document(title="Document 1", content="This is the first document.")
writer.add_document(title="Document 2", content="This is the second document.")
writer.commit()


In [54]:
# Indexing documents
writer = ix.writer()
# Add documents to index (replace with your actual PDF titles and content)
for i, filename in enumerate(os.listdir(pdf_directory)):
    if filename.endswith(".pdf"):
        with open(os.path.join(pdf_directory, filename), 'rb') as file:
            reader = PdfReader(file)
            content = ""
            for page in reader.pages:
                content += page.extract_text() + "\n"
            writer.add_document(title=filename, content=content)
writer.commit()

In [55]:
from whoosh.qparser import QueryParser
from whoosh.index import open_dir # import the open_dir function

ix = open_dir("indexdir") # Call the correct function
with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse("document")
    results = searcher.search(query)
    for result in results:
        print(result['title'])

Document 1
Document 2


In [56]:
from nltk.corpus import wordnet

# Get synonyms for query expansion
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return set(synonyms)

query = "happy"
expanded_query = query + " " + " ".join(get_synonyms(query))
print(expanded_query)


happy happy felicitous well-chosen glad


In [57]:
!pip install sentence_transformers

In [58]:
from sentence_transformers import SentenceTransformer, util

# Load BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Semantic search
query = "What is AI?"
query_embedding = model.encode(query)

documents = ["Document about artificial intelligence.", "Text related to machine learning."]
doc_embeddings = model.encode(documents)

# Find best match
hits = util.semantic_search(query_embedding, doc_embeddings)
print(hits)


[[{'corpus_id': 0, 'score': 0.5344616174697876}, {'corpus_id': 1, 'score': 0.22655534744262695}]]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [59]:
from whoosh import query

ix = open_dir("indexdir")
with ix.searcher() as searcher:
    fuzzy_query = query.FuzzyTerm("content", "document", maxdist=1)
    results = searcher.search(fuzzy_query)
    for result in results:
        print(result['title'])


Document 1
Document 2


In [60]:
!pip install streamlit
import streamlit as st
from whoosh.qparser import QueryParser
from whoosh.index import open_dir

# Streamlit app for search engine
st.title("Search Engine")

query = st.text_input("Enter search query:")

if query:
    ix = open_dir("indexdir")
    with ix.searcher() as searcher:
        parser = QueryParser("content", ix.schema)
        parsed_query = parser.parse(query)
        results = searcher.search(parsed_query)

        for result in results:
            st.write(result['title'])

2024-10-07 15:00:31.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 15:00:31.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 15:00:31.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 15:00:31.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 15:00:31.034 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 15:00:31.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 15:00:31.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 15:00:31.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [61]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have predicted and actual results
y_true = [1, 0, 1, 1, 0]
y_pred = [1, 0, 1, 0, 0]

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Precision: {precision}, Recall: {recall}, F1-Score: {f1}')


Precision: 1.0, Recall: 0.6666666666666666, F1-Score: 0.8


In [62]:
!pip install streamlit pyngrok joblib


In [63]:
%%writefile app.py
import streamlit as st
from whoosh.index import open_dir
from whoosh.qparser import QueryParser
from PyPDF2 import PdfReader
import os
from sentence_transformers import SentenceTransformer, util

pdf_directory = '/content/drive/My Drive/Colab Notebooks/MIniDataSet'
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def main():
    global pdf_directory
    st.title(" Breast Cancer Research Catalogue")
    query = st.text_input("Enter your search query:")

    if st.button("Search"):
        if query:
            ix = open_dir("indexdir")
            with ix.searcher() as searcher:
                parser = QueryParser("content", ix.schema)
                parsed_query = parser.parse(query)
                results = searcher.search(parsed_query)

                st.write("Search Results:")
                if results:
                    for result in results:
                        with st.expander(result['title']):
                            pdf_path = os.path.join(pdf_directory, result['title'])
                            with open(pdf_path, 'rb') as file:
                                reader = PdfReader(file)
                                pdf_text = ""
                                for page in reader.pages:
                                    pdf_text += page.extract_text()

                                sentences = pdf_text.split(". ")
                                query_embedding = model.encode(query)
                                sentence_embeddings = model.encode(sentences)
                                hits = util.semantic_search(query_embedding, sentence_embeddings, top_k=3)

                                for hit in hits[0]:
                                    # Get the index of the most similar sentence
                                    sentence_index = hit['corpus_id']

                                    # Display context: sentences before and after
                                    start_index = max(0, sentence_index - 2)  # 2 sentences before
                                    end_index = min(len(sentences), sentence_index + 3)  # 2 sentences after + the main sentence
                                    context = ". ".join(sentences[start_index:end_index])

                                    st.write(context)

                else:
                    st.write("No results found.")
        else:
            st.write("Please enter a query to search.")

if __name__ == '__main__':
    main()

Overwriting app.py


In [64]:
from pyngrok import ngrok
import os

# Kill any existing Streamlit processes
os.system('pkill streamlit')

# Run the Streamlit app
get_ipython().system_raw('streamlit run app.py &')

# Create a public URL using ngrok
!ngrok authtoken 2mhmVgSRpnqiFyZL3WoY4j0EA7H_6MuP3KYnr2QCVN3f2tYZ
url = ngrok.connect(addr='http://localhost:8501')
print(f"Streamlit app is live at: {url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://5961-34-148-188-236.ngrok-free.app" -> "http://localhost:8501"
